Jupyterlab dependencies, in addition to the normal ones:
```
conda install -c conda-forge jupyterlab nodejs
jupyter labextension install @jupyter-widgets/jupyterlab-manager ipyevents
```

To use `aggdraw` successfully in Windows, clone
https://github.com/ejeschke/aggdraw/ repository, follow
https://stackoverflow.com/questions/17770413/aggdraw-cannot-load-font-no-text-renderer,
and then install the `vectorized-drawing` branch.

To use `aggdraw` successfully in non-Windows:
```
conda install -c conda-forge freetype=2.8.1 aggdraw
```

For Windows users, `conda install -c conda-forge nodejs` will fail with `IOError`. To get around that, install `nodejs` from https://nodejs.org (the LTS version) and `yarn` from https://yarnpkg.com (the stable version).

In [53]:
# Use this in the future?!
#from stginga.jupyterw.widget import ImageWidget

"""
Dev notes
=========

GUI interactions:

* Right clicking should do ds9-style stretch adjustment. (*not* the same as
  the ``stretch`` property - here I mean "brightness/contrast" adjustment
  within the bounds of a given stretch)
* The user should be able to pan the view interactively.  This can be via
  middle clicking on the new center, click-and-drag, or scrolling (i.e. with
  touchpad a la what ginga does). The properties ``click_drag``,
  ``click_center``, and ``scroll`` can turn on/off these options
  (as does the "selection" mode).
* Zooming - if ``scroll_pan`` is False (probably the default), zooming is via
  the scroll wheel.
* "Selection mode" - see ``select_points`` method.
* If the user provides an NDData or fits input (assuming the fits file has
  valid WCS), if the cursor is not turned off it shows both the pixel
  coordinates and the WCS coordinates under the cursor.

Initially, *no* keyboard shortcuts should be implemented.  Eventually there
should be a clear mapping from keyboard shortcuts to methods, but until the
methods are stabilized, the keyboard shortcuts should be avoided.

Other requirements:

* Should be able to hanle ~4k x 4k images without significant performance
  lagging.
* Should be able to handle ~1000x markers without significant performance
  degredation.
* Stretch goal: hould be able to handle ~10k x 10k images acceptable
* Extra-stretchy goal: handle very large datasets using a "tiling" approach.
  This will presumably require different ``load_*`` functions, and more
  cleverness on the JS side.

A few more notes:

* We should be subclassing some kind of ipywidget,
  likely Box is the best choice.
* If we do that, then _repr_html is unnecessary (and undesirable), because
  the widget machinery will take care of it.
* Really like to avoid middle-click interactions, or at least I would like
  them to have an alias that works on a trackpad or a two-button mouse.
* I'd like a little more flexibility in adding markers (i.e., not necessarily
  require the use of a table, though that should be one way to do it).
* I also think we need at least minimal ability to change/set marker color,
  shape very early on.

"""
# STDLIB
import functools

# THIRD-PARTY
import numpy as np
from astropy.coordinates import SkyCoord
from astropy.io import fits
from astropy.table import Table

# Jupyter widgets
import ipywidgets as ipyw
from IPython.display import display

# Ginga
from ginga.AstroImage import AstroImage
from ginga.canvas.CanvasObject import drawCatalog
from ginga.web.jupyterw.ImageViewJpw import EnhancedCanvasView
from ginga.util.wcs import raDegToString, decDegToString


class ImageWidget(object):
    """
    Image widget for Jupyter notebook using Ginga viewer.

    .. todo:: Any property passed to constructor has to be valid keyword.

    Parameters
    ----------
    logger : obj or `None`
        Ginga logger. For example::

            from ginga.misc.log import get_logger
            logger = get_logger('my_viewer', log_stderr=False,
                                log_file='ginga.log', level=40)

    width, height : int
        Dimension of Jupyter notebook's image widget.
        
    use_opencv : bool
        Let Ginga use ``opencv`` to speed up image transformation;
        e.g., rotation and mosaic. If this is enabled and you
        do not have ``opencv``, you will see ``ImportError``.

    """
    def __init__(self, logger=None, width=500, height=500, use_opencv=True):
        
        # TODO: Is this the best place for this?
        if use_opencv:
            from ginga import trcalc
            trcalc.use('opencv')
        
        self._viewer = EnhancedCanvasView(logger=logger)
        self._is_marking = False
        self._click_center = False

        self._jup_img = ipyw.Image(format='jpeg', width=width, height=height)
        self._viewer.set_widget(self._jup_img)

        # enable all possible keyboard and pointer operations
        self._viewer.get_bindings().enable_all(True)

        # enable draw
        self.dc = drawCatalog
        self.canvas = self.dc.DrawingCanvas()
        self.canvas.enable_draw(True)
        self.canvas.enable_edit(True)

        # Marker
        self.marker = {'type': 'circle', 'color': 'cyan', 'radius': 20}
        self._marktag = 'marktag'
        
        # coordinates display
        self._jup_coord = ipyw.HTML('Coordinates show up here')
        # This needs ipyevents 0.3.1 to work
        self._viewer.add_callback('cursor-changed', self._mouse_move_cb)
        self._viewer.add_callback('cursor-down', self._mouse_click_cb)
        
        # Define a callback that shows the output of a print
        self.print_out = ipyw.Output()
        
        self._cursor = 'bottom'
        self._widget = ipyw.VBox([self._jup_img, self._jup_coord])
        
    @property
    def logger(self):
        """Logger for this widget."""
        return self._viewer.logger
        
    def _mouse_move_cb(self, viewer, button, data_x, data_y):
        """
        Callback to display position in RA/DEC deg.
        """
        if self.cursor is None:  # no-op
            return
        
        image = viewer.get_image()
        if image is not None:
            ix = int(data_x + 0.5)
            iy = int(data_y + 0.5)
            try:
                imval = viewer.get_data(ix, iy)
            except Exception:
                imval = 'N/A'
            
            # Same as setting pixel_coords_offset=1 in general.cfg
            val = 'X: {:.2f}, Y:{:.2f}'.format(data_x + 1, data_y + 1)
            if image.wcs.wcs is not None:
                ra, dec = image.pixtoradec(data_x, data_y)
                val += ' (RA: {}, DEC: {})'.format(
                    raDegToString(ra), decDegToString(dec))
            val += ', value: {}'.format(imval)
            self._jup_coord.value = val

    def _mouse_click_cb(self, viewer, event, data_x, data_y):
        """
        Callback to handle mouse clicks.
        """        
        if self.is_marking:
            objs = []
            try:
                c_mark = viewer.canvas.get_object_by_tag(self._marktag)
            except Exception:  # Nothing drawn yet
                pass
            else:  # Add to existing marks
                objs = c_mark.objects
                viewer.canvas.delete_object_by_tag(self._marktag)

            # NOTE: By always using CompoundObject, marker handling logic
            # is simplified.
            obj = self.marker(x=data_x, y=data_y)
            objs.append(obj)
            self._marktag = viewer.canvas.add(self.dc.CompoundObject(*objs))
            
            with self.print_out:
                print('Selected {} {}'.format(obj.x, obj.y))
            
        elif self.click_center:
            self.center_on((data_x, data_y))
            
            with self.print_out:
                print('Centered on X={} Y={}'.format(data_x + 1, data_y + 1))

    def _repr_html_(self):
        """
        Show widget in Jupyter notebook.
        """
        return display(self._widget)
    
    def load_fits(self, fitsorfn, numhdu=None, memmap=None):
        """
        Load a FITS file into the viewer.

        Parameters
        ----------
        fitsorfn : str or HDU
            Either a file name or an HDU (*not* an HDUList).
            If file name is given, WCS in primary header is automatically
            inherited. If a single HDU is given, WCS must be in the HDU
            header.

        numhdu : int or `None`
            Extension number of the desired HDU.
            If `None`, it is determined automatically.
            
        memmap : bool or `None`
            Memory mapping.
            If `None, it is determined automatically.

        """
        if isinstance(fitsorfn, str):
            image = AstroImage(logger=self.logger, inherit_primary_header=True)
            image.load_file(fitsorfn, numhdu=numhdu, memmap=memmap)
            self._viewer.set_image(image)

        elif isinstance(fitsorfn, (fits.ImageHDU, fits.CompImageHDU,
                                   fits.PrimaryHDU)):
            self._viewer.load_hdu(fitsorfn)

    def load_nddata(self, nddata):
        """
        Load an ``NDData`` object into the viewer.

        .. todo:: Add flag/masking support, etc.

        Parameters
        ----------
        nddata : `~astropy.nddata.NDData`
            ``NDData`` with image data and WCS.

        """
        from ginga.util.wcsmod.wcs_astropy import AstropyWCS
        
        image = AstroImage(logger=self.logger)
        image.set_data(nddata.data)
        _wcs = AstropyWCS(self.logger)
        _wcs.load_header(nddata.wcs.to_header())
        try:
            image.set_wcs(_wcs)
        except Exception as e:
            print('Unable to set WCS from NDData: {}'.format(str(e)))
        self._viewer.set_image(image)

    def load_array(self, arr):
        """
        Load a 2D array into the viewer.

        .. note:: Use :meth:`load_nddata` for WCS support.

        Parameters
        ----------
        arr : array-like
            2D array.

        """
        self._viewer.load_data(arr)

    def center_on(self, point, pixel_coords_offset=1):
        """
        Centers the view on a particular point.
        
        Parameters
        ----------
        point : tuple or `~astropy.coordinates.SkyCoord`
            If tuple of ``(X, Y)`` is given, it is assumed
            to be in data coordinates.
        
        pixel_coords_offset : {0, 1}
            Data coordinates provided are n-indexed,
            where n is the given value.
            This is ignored if ``SkyCoord`` is provided.
        
        """
        if isinstance(point, SkyCoord):
            self._viewer.set_pan(point.ra.deg, point.dec.deg, coord='wcs')
        else:
            self._viewer.set_pan(*(np.asarray(point) - pixel_coords_offset))

    def offset_to(self, dx, dy, skycoord_offset=False):
        """
        Move the center to a point that is given offset
        away from the current center.
        
        Parameters
        ----------
        dx, dy : float
            Offset value. Unit is assumed based on
            ``skycoord_offset``.
            
        skycoord_offset : bool
            If `True`, offset must be given in degrees.
            Otherwise, they are in pixel values.
        
        """
        if skycoord_offset:
            coord = 'wcs'
        else:
            coord = 'data'
        
        pan_x, pan_y = self._viewer.get_pan(coord=coord)
        self._viewer.set_pan(pan_x + dx, pan_y + dy, coord=coord)

    @property
    def zoom_level(self):
        """
        Zoom level:

        * 1 means real-pixel-size.
        * 2 means zoomed in by a factor of 2.
        * 0.5 means zoomed out by a factor of 2.

        """
        return self._viewer.get_scale()

    def zoom(self, val):
        """
        Zoom in or out by the given factor.

        Parameters
        ----------
        val : int
            The zoom level to zoom the image.
            See `zoom_level`.

        """
        self._viewer.scale_to(val, val)

    @property
    def is_marking(self):
        """
        `True` if in marking mode, `False` otherwise.
        Marking mode means a mouse click adds a new marker.
        This does not affect :meth:`add_markers`.
        """
        return self._is_marking
    
    @is_marking.setter
    def is_marking(self, val):
        if not isinstance(val, bool):
            raise ValueError('Must be True or False')
        elif self.click_center and val:
            raise ValueError('Cannot set to True while in click-center mode')
        self._is_marking = val

    def stop_marking(self, clear_markers=True):
        """
        Stop marking mode, with option to clear markers, if desired.

        Parameters
        ----------
        clear_markers : bool
            If ``clear_markers`` is `False`, existing markers are
            retained until :meth:`reset_markers` is called.
            Otherwise, they are erased.
        """
        self.is_marking = False
        if clear_markers:
            self.reset_markers()
        
    @property
    def marker(self):
        """
        Marker to use.
        
        .. todo:: Add more examples.
        
        Marker can be set as follows::
            
            {'type': 'circle', 'color': 'cyan', 'radius': 20}
        
        """
        return self._marker
    
    @marker.setter
    def marker(self, val):
        marker_type = val.pop('type')
        if marker_type == 'circle':
            self._marker = functools.partial(self.dc.Circle, **val)
        else:  # TODO: Implement more shapes
            raise NotImplementedError(
                'Marker type "{}" not supported'.format(marker_type))
        
    def get_markers(self, x_colname='x', y_colname='y',
                    pixel_coords_offset=1,
                    skycoord_colname='coord'):
        """
        Return the locations of existing markers.

        Parameters
        ----------
        x_colname, y_colname : str
            Column names for X and Y data coordinates.
            Coordinates retured are 0- or 1-indexed, depending
            on ``pixel_coords_offset``.
            
        pixel_coords_offset : {0, 1}
            Data coordinates returned are n-indexed,
            where n is the given value.
        
        skycoord_colname : str
            Column name for ``SkyCoord``, which contains
            sky coordinates associated with the active image.
            This is ignored if image has no WCS.

        Returns
        -------
        markers_table : `~astropy.table.Table` or `None`
            Table of markers, if any, or `None`.

        """
        try:
            c_mark = self._viewer.canvas.get_object_by_tag(self._marktag)
        except Exception as e:  # No markers
            self.logger.warning(str(e))
            return
        
        image = self._viewer.get_image()
        xy_col = []

        if image.wcs.wcs is None:  # Do not include SkyCoord column
            include_skycoord = False
        else:
            include_skycoord = True
            radec_col = []
 
        # Extract coordinates from markers
        for obj in c_mark.objects:
            if obj.coord == 'data':
                xy_col.append([obj.x, obj.y])
                if include_skycoord:
                    radec_col.append([np.nan, np.nan])
            elif not include_skycoord:  # marker in WCS but image has none
                self.logger.warning(
                    'Skipping ({},{}); image has no WCS'.format(obj.x, obj.y))
            else:  # wcs
                xy_col.append([np.nan, np.nan])
                radec_col.append([obj.x, obj.y])

        # Convert to numpy arrays
        xy_col = np.asarray(xy_col)  # [[x0, y0], [x1, y1], ...]

        if include_skycoord:
            radec_col = np.asarray(radec_col)  # [[ra0, dec0], [ra1, dec1], ...]
            
            # Fill in X,Y from RA,DEC
            mask = np.isnan(xy_col[:, 0])  # One bool per row
            if np.any(mask):
                xy_col[mask] = image.wcs.wcspt_to_datapt(radec_col[mask])
            
            # Fill in RA,DEC from X,Y
            mask = np.isnan(radec_col[:, 0])
            if np.any(mask):
                radec_col[mask] = image.wcs.datapt_to_wcspt(xy_col[mask])
            
            sky_col = SkyCoord(radec_col[:, 0], radec_col[:, 1], unit='deg')
    
        # Convert X,Y from 0-indexed to 1-indexed
        if pixel_coords_offset != 0:
            xy_col += pixel_coords_offset
    
        # Build table
        if include_skycoord:
            markers_table = Table(
                [xy_col[:, 0], xy_col[:, 1], sky_col],
                names=(x_colname, y_colname, skycoord_colname))
        else:
            markers_table = Table(xy_col.T, names=(x_colname, y_colname))
    
        return markers_table
    
    def add_markers(self, table, x_colname='x', y_colname='y',
                    pixel_coords_offset=1,
                    skycoord_colname='coord', use_skycoord=False):
        """
        Creates markers in the image at given points.

        .. todo::
        
            Later enhancements to include more columns
            to control size/style/color of marks,

        Parameters
        ----------
        table : `~astropy.table.Table`
            Table containing marker locations.

        x_colname, y_colname : str
            Column names for X and Y.
            Coordinates can be 0- or 1-indexed, as
            given by ``pixel_coords_offset``.

        pixel_coords_offset : {0, 1}
            Data coordinates provided are n-indexed,
            where n is the given value.
            This is ignored if ``use_skycoord=True``.

        skycoord_colname : str
            Column name with ``SkyCoord`` objects.

        use_skycoord : bool
            If `True`, use ``skycoord_colname`` to mark.
            Otherwise, use ``x_colname`` and ``y_colname``.
            
        """
        # TODO: Resolve https://github.com/ejeschke/ginga/issues/672
        # Extract coordinates from table.
        # They are always arrays, not scalar.
        if use_skycoord:
            image = self._viewer.get_image()
            if image is None:
                raise ValueError('Cannot get image from viewer')
            if image.wcs.wcs is None:
                raise ValueError(
                    'Image has no valid WCS, '
                    'try again with use_skycoord=False')
            coord_type = 'wcs'
            coord_val = table[skycoord_colname]
            coord_x = coord_val.ra.deg
            coord_y = coord_val.dec.deg
        else:  # Use X,Y
            coord_type = 'data'
            coord_x = table[x_colname].data
            coord_y = table[y_colname].data
            # Convert data coordinates from 1-indexed to 0-indexed
            if pixel_coords_offset != 0:
                coord_x -= pixel_coords_offset
                coord_y -= pixel_coords_offset

        # Prepare canvas and retain existing marks
        objs = []
        try:
            c_mark = self._viewer.canvas.get_object_by_tag(self._marktag)
        except Exception:
            pass
        else:
            objs = c_mark.objects
            self._viewer.canvas.delete_object_by_tag(self._marktag)
        
        # TODO: Test to see if we can mix WCS and data on the same canvas
        objs += [self.marker(x=x, y=y, coord=coord_type)
                 for x, y in zip(coord_x, coord_y)]
        self._marktag = self._viewer.canvas.add(self.dc.CompoundObject(*objs))

    # TODO: Future work?
    def remove_markers(self, arr):
        """
        Remove some but not all of the markers.
        
        Parameters
        ----------
        arr : ``SkyCoord`` or array-like
            Sky coordinates or 2xN array.
            
        """
        # NOTE: How to match? Use np.isclose?
        #       What if there are 1-to-many matches?
        return NotImplementedError
        
    def reset_markers(self):
        """
        Delete all markers.
        """
        try:
            self._viewer.canvas.delete_object_by_tag(self._marktag)
        except Exception:
            pass

    @property
    def stretch_options(self):
        """
        List all available options for image stretching.
        """
        return self._viewer.get_color_algorithms()
    
    @property
    def stretch(self):
        """
        The image stretching algorithm in use.
        """
        return self._viewer.rgbmap.dist
    
    # TODO: Possible to use astropy.visualization directly?
    @stretch.setter
    def stretch(self, val):
        valid_vals = self.stretch_options
        if val not in valid_vals:
            raise ValueError('Value must be one of: {}'.format(valid_vals))
        self._viewer.set_color_algorithm(val)

    @property
    def autocut_options(self):
        """
        List all available options for image auto-cut.
        """
        return w._viewer.get_autocut_methods()
        
    @property
    def cuts(self):
        """
        Current image cut levels.
        To set new cut levels, either provide a tuple of
        ``(low, high)`` values or one of the options from
        `autocut_options`.
        """
        return self._viewer.get_cut_levels()
    
    # TODO: Possible to use astropy.visualization directly?
    @cuts.setter
    def cuts(self, val):
        if isinstance(val, str):  # Autocut
            valid_vals = self.autocut_options
            if val not in valid_vals:
                raise ValueError('Value must be one of: {}'.format(valid_vals))
            self._viewer.set_autocut_params(val)
        else:  # (low, high)
            self._viewer.cut_levels(val[0], val[1])

    @property
    def cursor(self):
        """
        Show or hide cursor information (X, Y, WCS).
        Acceptable values are 'top', 'bottom', or `None`.
        """
        return self._cursor

    @cursor.setter
    def cursor(self, val):
        if val is None:
            self._widget = self._jup_img
        elif val == 'top':
            self._widget = ipyw.VBox([self._jup_coord, self._jup_img])
        elif val == 'bottom':
            self._widget = ipyw.VBox([self._jup_img, self._jup_coord])
        else:
            raise NotImplementedError
        self._cursor = val
        
    @property
    def click_center(self):
        """
        Settable.
        If True, middle-clicking can be used to center.  If False, that
        interaction is disabled.

        In the future this might go from True/False to being a selectable
        button. But not for the first round.
        """
        return self._click_center

    @click_center.setter
    def click_center(self, val):
        if not isinstance(val, bool):
            raise ValueError('Must be True or False')
        elif self.is_marking and val:
            raise ValueError('Cannot set to True while in marking mode')
        self._click_center = val
        
    # TODO: Awaiting https://github.com/ejeschke/ginga/issues/674
    @property
    def click_drag(self):
        """
        Settable.
        If True, the "click-and-drag" mode is an available interaction for
        panning.  If False, it is not.

        Note that this should be automatically made `False` when selection mode
        is activated.
        """
        raise NotImplementedError
        
    @property
    def scroll_pan(self):
        """
        Settable.
        If True, scrolling moves around in the image.  If False, scrolling
        (up/down) *zooms* the image in and out.
        """
        raise NotImplementedError

    # https://github.com/ejeschke/ginga/pull/665
    def save(self, filename):
        """
        Save out the current image view to given PNG filename.
        """
        self._viewer.save_rgb_image_as_file(filename)


In [54]:
from ginga.misc.log import get_logger

logger = get_logger('my viewer', log_stderr=True,
                    log_file=None, level=30)

In [55]:
w = ImageWidget(logger=logger)

In [56]:
#filename = '../../../stginga/test_data/candels_big_mosaic.fits'
#numhdu = 0
#data_unit = None

filename = '../../../stginga/test_data/jb5g05ubq_flt.fits'
numhdu = 4
data_unit = 'electron'

# Loads a FITS file
w.load_fits(filename, numhdu=numhdu)

# Loads NDData
#from astropy.nddata import CCDData
#ccd = CCDData.read(filename, hdu=numhdu, unit=data_unit)
#w.load_nddata(ccd)

# Loads array (no WCS)
#from astropy.io import fits
#with fits.open(filename, memmap=False) as pf:
#    arr = pf[numhdu].data.copy()
#w.load_array(arr)

Ginga key bindings documented at http://ginga.readthedocs.io/en/latest/quickref.html . Note that not all documented bindings would work here. Please use an alternate binding, if available, if the chosen one is not working.

Here are the ones that worked during testing with Firefox 52.8.0 on RHEL7 64-bit:

Key | Action | Notes
--- | --- | ---
`+` | Zoom in |
`-` | Zoom out |
Number (0-9) | Zoom in to specified level | 0 = 10
Shift + number | Zoom out to specified level | Numpad does not work
` (backtick) | Reset zoom |
Space > `q` > arrow | Pan |
ESC | Exit mode (pan, etc) |
`c` | Center image
Space > `d` > up/down arrow | Cycle through color distributions
Space > `d` > Shift + `d` | Go back to linear color distribution
Space > `s` > Shift + `s` | Set cut level to min/max
Space > `s` > Shift + `a` | Set cut level to 0/255 (for 8bpp RGB images)
Space > `s` > up/down arrow | Cycle through cuts algorithms
Space > `l` | Toggle no/soft/normal lock |

**TODO: Check out Contrast Mode next**

A viewer will be shown after running the next cell.
In Jupyter Lab, you can split it out into a separate view by right-clicking on the viewer and then select
"Create New View for Output". Then, you can drag the new
"Output View" tab, say, to the right side of the workspace. Both viewers are connected to the same events.

In [57]:
w

This next cell captures print outputs. You can pop it out like the viewer above. It is very convenient for debugging purpose.

In [58]:
# Capture print outputs from the widget
display(w.print_out)

Output()

The following cell changes the visibility or position of the cursor info bar. For the new setting to take effect, you need to re-run the cell that calls `w` above.

In [ ]:
w.cursor = 'top'  # 'top', 'bottom', None
print(w.cursor)

The rest of the calls demonstrate how the widget API works. Comment/uncomment as needed. Feel free to experiment.

In [39]:
# Programmatically center to (X, Y) on viewer
#w.center_on((1, 1))

# Programmatically offset w.r.t. current center
#w.offset_to(4096, 2048)

# Programmatically center to SkyCoord on viewer
#w.center_on(SkyCoord('00h14m28.28s', '-30d23m42.66s', frame='icrs'))

# Programmatically offset (in degrees) w.r.t. SkyCoord center
#w.offset_to(0.001, 0.001, skycoord_offset=True)

# Show zoom level
#print(w.zoom_level)

# Programmatically zoom image on viewer
#w.zoom(2)

# Capture what viewer is showing and save RGB image
# Need https://github.com/ejeschke/ginga/pull/665 to work
#w.save('test.png')

In [ ]:
# Get all available image stretch options
print(w.stretch_options)

# Get image stretch algorithm in use
print(w.stretch)

# Change the stretch
w.stretch = 'linear'
print(w.stretch)

In [ ]:
# Get all available image cuts options
print(w.autocut_options)

# Get image cuts algorithm in use
print(w.cuts)

# Change the cuts by providing explicit low/high values
w.cuts = (0, 100)
print(w.cuts)

# Change the cuts with an autocut algorithm
w.cuts = 'zscale'
print(w.cuts)

In [64]:
# This enables click to center.
#w.click_center = True

# Turn it back off so marking (next cell) can be done.
w.click_center = False

In [59]:
# This enables/disabled marking mode.
# Set to True, click on viewer to mark.
# When done, set back to False.
w.is_marking = True
#w.is_marking = False
print(w.is_marking)

True


In [60]:
# Get table of markers
markers_table = w.get_markers()
print(markers_table)

# For sanity check with written values while marking
print()
for c in markers_table['coord'][:2]:
    print(c.to_string('hmsdms'))

        x             y                    coord                 
                                          deg,deg                
------------------ ------- --------------------------------------
           1082.58  681.02 3.6090563354665894,-30.378933369990555
           2507.64  574.55 3.5896481022734865,-30.369046606433923
1254.5700000000002 1360.79  3.6120849724158393,-30.36960772587492

00h14m26.1735s -30d22m44.1601s
00h14m21.5155s -30d22m08.5678s


In [ ]:
# Erase markers
w.reset_markers()

In [ ]:
# Programmatically re-mark from table using X, Y.
# To be fancy, first 2 points marked differently.
w.marker = {'type': 'circle', 'color': 'red', 'radius': 50}
w.add_markers(markers_table[:2])
w.marker = {'type': 'circle', 'color': 'cyan', 'radius': 20}
w.add_markers(markers_table[2:])

# TODO: Make this work
# Programmatically re-mark from table using SkyCoord
#w.add_markers(markers_table, use_skycoord=True)

In [ ]:
# Stop marking AND clear markers
w.stop_marking(clear_markers=True)
print(w.is_marking)

In [ ]:
import numpy as np
from astropy.table import Table

# Generate random "stars" to mark
max_stars = 1000
dpix = 20
img = w._viewer.get_image()
bad_locs = np.random.randint(dpix, high=img.shape[1] - dpix, size=[max_stars, 2])

# Only want those not near the edges
mask = ((dpix < bad_locs[:, 0]) & (bad_locs[:, 0] < img.shape[0] - dpix) &
        (dpix < bad_locs[:, 1]) & (bad_locs[:, 1] < img.shape[1] - dpix))

# Put them in table
locs = bad_locs[mask]
t = Table([locs[:, 1], locs[:, 0]], names=('x', 'y'))
print(t)

In [ ]:
# Mark those stars
w.add_markers(t)

In [ ]:
# Define a function to control marker display
def show_circles(n):
    """
    Show and hide circles instead of reconstructing them.
    """
    w.reset_markers()
    t2show = t[:n]
    w.add_markers(t2show)

In [ ]:
# import ipywidgets as ipyw
from IPython.display import display
from ipywidgets import interact

# Initialize and display the slider right below the viewer.
interact(
    show_circles,
    n=ipyw.IntSlider(min=0,max=len(t),step=1,value=0));